In [1]:
import pandas as pd
import tensorflow as tf
from pathlib import Path
import numpy as np

2023-10-26 09:18:30.869969: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-26 09:18:31.453552: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
ALL_RECS = list(DATA_PATH.glob("*.tfrecords"))

In [3]:
BATCH_SIZE = 1024

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'file_name': tf.io.FixedLenFeature([], tf.int64),
    'hospital_expire_flag': tf.io.FixedLenFeature([], tf.float32),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    return ecg_data, example['file_name']

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
model = tf.keras.models.load_model("data/models/resnet-age.keras")

2023-10-26 09:18:42.608584: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 507 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:18:00.0, compute capability: 7.0


In [6]:
all_recs = get_dataset(ALL_RECS)
results = {}
for batch in all_recs:
    preds = model.predict(batch[0])
    for age, study in zip(preds, batch[1]):
        results[study.numpy()] = age[0]

7/7 [==============================] - 1s 121ms/step


In [21]:
results2 = {}
for x in results.keys():
    results2[int(x)] =  float(results[x])

In [22]:
from json import dump
with open("data/resnet-age-scores.json", "w") as fh:
    dump(results2, fh)